In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np

import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.transforms as mtrans
import seaborn as sns

pd.set_option('display.max_rows', 150)

In [ ]:
with open(os.path.join(os.curdir, 'data', 'tech_by_start_date.json')) as f:
    tech_by_start_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_track_by_date.json')) as f:
    num_of_track_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'prize_of_track_by_date.json')) as f:
    prize_of_track_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_dev_subtrack_by_dt.json')) as f:
    num_of_dev_subtrack_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'prize_of_dev_subtrack_by_dt.json')) as f:
    prize_of_dev_subtrack_by_dt = json.load(f)

with open(os.path.join(os.curdir, 'data', 'number_of_challenges_by_project.json')) as f:
    num_of_challenges_by_project = json.load(f)

df_tech_by_dt = pd.DataFrame.from_dict(tech_by_start_dt, orient='index').fillna(0).astype(int).drop(columns='other')
df_cha_by_proj = pd.DataFrame(num_of_challenges_by_project).set_index('project_id')
df_prz_of_track_by_dt = pd.DataFrame(prize_of_track_by_dt).set_index('date')
df_num_of_track_by_dt = pd.DataFrame(num_of_track_by_dt).set_index('date')
df_prz_of_dev_subtrack_by_dt = pd.DataFrame(prize_of_dev_subtrack_by_dt).set_index('date')
df_num_of_dev_subtrack_by_dt = pd.DataFrame(num_of_dev_subtrack_by_dt).set_index('date').astype(int)

df_tech_by_dt.index = pd.to_datetime(df_tech_by_dt.index)
df_prz_of_track_by_dt.index = pd.to_datetime(df_prz_of_track_by_dt.index)
df_num_of_track_by_dt.index = pd.to_datetime(df_num_of_track_by_dt.index)
df_prz_of_dev_subtrack_by_dt.index = pd.to_datetime(df_prz_of_dev_subtrack_by_dt.index)
df_num_of_dev_subtrack_by_dt.index = pd.to_datetime(df_num_of_dev_subtrack_by_dt.index)

df_tech_by_dt.sort_index(inplace=True)
df_prz_of_dev_subtrack_by_dt.sort_index(inplace=True)
df_num_of_dev_subtrack_by_dt.sort_index(inplace=True)
df_prz_of_track_by_dt.sort_index(inplace=True)
df_num_of_track_by_dt.sort_index(inplace=True)

In [ ]:
# Util functions

def config_xtick_and_label(ax):
    """ Format the labels of xaxis where the x is a time axis."""
    year_locator = mdates.YearLocator()
    month_locator = mdates.MonthLocator()
    year_fmt = mdates.DateFormatter('%Y')
#     month_fmt = mdates.DateFormatter('%b')
    
    ax.xaxis_date()
    ax.xaxis.set_major_locator(year_locator)
    ax.xaxis.set_major_formatter(year_fmt)
    ax.xaxis.set_minor_locator(month_locator)
#     ax.xaxis.set_minor_formatter(month_fmt)
    ax.set_xlim(datetime.datetime.strptime('2014', '%Y'), datetime.datetime.strptime('2021', '%Y'))

In [ ]:
# Get all the project with more than 10 challenges under it

df_filtered_proj = \
    df_cha_by_proj.loc[df_cha_by_proj.number_of_challenges >= 10]\
    .sort_values(by='number_of_challenges', ascending=False)

fig = plt.figure(figsize=(8.5, 6), dpi=200)

with sns.axes_style('dark', {'xtick.bottom': True}):
    ax0 = fig.add_subplot(2, 1, 1)
    
    sns.distplot(df_filtered_proj.number_of_challenges, bins=20, kde=False, rug=True, ax=ax0)
    
    ax0.set_title('Number of challenges under one project\nDistribution - histogram')
    ax0.set_ylabel('Frequency')
    ax0.set_xlabel('Number of challenges')
    
    ax0.set_xticks(list(range(10, 220, 10)))
    ax0.set_xticklabels(labels=list(range(10, 220, 10)), rotation=315)
    ax0.set_ylim(bottom=0, top=200)
    
    ax0.grid(True, axis='y')
    
    for hist in ax0.patches:
        count = int(hist.get_height())
        x = hist.get_x() + hist.get_width() / 2
        y = hist.get_height()
        if count != 0:
            ax0.annotate(
                f'{count}', 
                xy=(x, y), 
                xytext=(0, 3), 
                horizontalalignment='center', 
                verticalalignment='bottom',
                textcoords='offset points'
            )

with sns.axes_style('darkgrid'):
    ax1 = fig.add_subplot(2, 1, 2)
    
    colors = {'primary': '#E93C4F', 'secondary': '#FFC24A'}
    
    meanlineprops = {'linestyle': '--', 'linewidth': 0.5, 'color': colors['secondary']}
    flierprops = {'marker': 'o', 'markerfacecolor': colors['primary'], 'markeredgewidth': 0.5, 'markersize': 2.5}
    sns.boxplot(
        x=df_cha_by_proj.loc[df_cha_by_proj.number_of_challenges > 10], 
        showmeans=True,
        meanline=True,
        
        color=colors['primary'],
        meanprops=meanlineprops,
        flierprops=flierprops,
        
        linewidth=0.8,
        width=0.618,
        
        ax=ax1
    )
    
    ax1.set_title('Number of challenges under one project\nDistribution - boxplot')
    ax1.set_xlabel('Number of challenges')

    ax1.set_xticks(list(range(20, 220, 20)))
    ax1.set_xticklabels(labels=list(range(20, 220, 20)))
    
fig.tight_layout()

In [ ]:
# Get the summary of technology terms frequency and choose the columns for plotting
tech_sum = df_tech_by_dt.sum(axis='index').sort_values(ascending=False)

colnames = [
    list(tech_sum[tech_sum >= 800].index),
    list(tech_sum[(400 <= tech_sum) & (tech_sum < 800)].index),
    list(tech_sum[(250 <= tech_sum) & (tech_sum < 400)].index),
    list(tech_sum[(150 <= tech_sum) & (tech_sum < 250)].index),
    list(tech_sum[(100 <= tech_sum) & (tech_sum < 150)].index)
]

title_dct = {
    0: 'Technologies tagged more than 800 times',
    1: 'Technologies tagged between 400 and 800 times',
    2: 'Technologies tagged between 250 and 400 times',
    3: 'Technologies tagged between 150 and 250 times',
    4: 'Technologies tagged between 100 and 150 times\nCumulative Summary'
}

fig_type_dct = {
    0: 'Cumulative Summary',
    1: 'Month by month'
}
    
df_tech_sum_by_month = df_tech_by_dt.groupby([df_tech_by_dt.index.year, df_tech_by_dt.index.month]).sum()
df_tech_sum_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_tech_sum_by_month.index.to_flat_index()])
df_tech_sum_by_month.drop(df_tech_sum_by_month.loc[df_tech_sum_by_month.index > '2020-03-31'].index, inplace=True)

fig, axes = plt.subplots(10, 1, figsize=(8.5, 40), dpi=200)

with sns.axes_style('ticks'):
    for idx, ax in enumerate(axes):
        
        ax_title = '{}{}'.format(f'{title_dct[idx // 2]}\n\n' if idx % 2 == 0 else '', fig_type_dct[idx % 2])
        data = df_tech_sum_by_month[colnames[idx // 2]].cumsum() if idx % 2 == 0 else df_tech_sum_by_month[colnames[idx // 2]]
        
        sns.despine(ax=ax)
        config_xtick_and_label(ax)
        ax.set_title(ax_title)
        ax.set_xlabel('Time')
        ax.set_ylabel('Frequency')

        sns.lineplot(data=data, dashes=False, ax=ax, alpha=0.6, legend='full', linewidth=0.8)
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, ncol=2, prop={'size': 8})
    
fig.tight_layout()

In [ ]:
df_track_sum_by_month = df_num_of_track_by_dt.groupby([df_num_of_track_by_dt.index.year, df_num_of_track_by_dt.index.month]).sum()
df_track_sum_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_track_sum_by_month.index.to_flat_index()])

df_track_prz_by_month = df_prz_of_track_by_dt.groupby([df_prz_of_track_by_dt.index.year, df_prz_of_track_by_dt.index.month]).sum()
df_track_prz_by_month.index = pd.to_datetime([f'{year}-{month}' for year, month in df_track_prz_by_month.index.to_flat_index()])

df_prz_per_cha_by_track = df_track_prz_by_month / df_track_sum_by_month
df_prz_per_cha_by_track.fillna(0)

fig = plt.figure(figsize=(8.5, 12), dpi=200)

with sns.axes_style('darkgrid', {'xtick.bottom': True}):
    ax0 = fig.add_subplot(3, 1, 1)
    sns.despine(ax=ax0)
    config_xtick_and_label(ax0)
    sns.lineplot(data=df_track_sum_by_month, dashes=False, ax=ax0, linewidth=0.75)

    ax1 = fig.add_subplot(3, 1, 2)
    sns.despine(ax=ax1)
    config_xtick_and_label(ax1)
    sns.lineplot(data=df_track_prz_by_month, dashes=False, ax=ax1, linewidth=0.75)
    
    ax2 = fig.add_subplot(3, 1, 3)
    sns.despine(ax=ax2)
    config_xtick_and_label(ax2)
    sns.lineplot(data=df_prz_per_cha_by_track, dashes=False, ax=ax2, linewidth=0.75)

In [ ]:
df_develop_track_num_prz = pd.concat([df_track_sum_by_month.develop, df_track_prz_by_month.develop], axis=1)
df_develop_track_num_prz.columns = ['cnt', 'prize']
df_develop_track_num_prz['prz_per_cha'] = df_develop_track_num_prz.prize / df_develop_track_num_prz.cnt

df_design_track_num_prz = pd.concat([df_track_sum_by_month.design, df_track_prz_by_month.design], axis=1)
df_design_track_num_prz.columns = ['cnt', 'prize']
df_design_track_num_prz['prz_per_cha'] = df_design_track_num_prz.prize / df_design_track_num_prz.cnt

df_data_science_track_num_prz = pd.concat([df_track_sum_by_month.data_science, df_track_prz_by_month.data_science], axis=1)
df_data_science_track_num_prz.columns = ['cnt', 'prize']
df_data_science_track_num_prz['prz_per_cha'] = df_data_science_track_num_prz.prize / df_data_science_track_num_prz.cnt

df_lst = [
    {'title': 'DEVELOP', 'df': df_develop_track_num_prz},
    {'title': 'DESIGN', 'df': df_design_track_num_prz},
    {'title': 'DATA_SCIENCE', 'df': df_data_science_track_num_prz}
]

fig = plt.figure(figsize=(8.5, 9), dpi=200)

with sns.axes_style('dark', {'xitck.bottom': True}):

    for i, df in enumerate(df_lst):
        ax0 = fig.add_subplot(3, 1, i + 1)
        ax1 = ax0.twinx()
        config_xtick_and_label(ax0)
        
        sns.lineplot(data=df['df'].cnt, label='Number of challenges', ax=ax0, color='#F78562', legend=False, linewidth=0.75)
        sns.lineplot(data=df['df'].prz_per_cha, label='Prize per challenge', ax=ax1, color='#34618F', legend=False, linewidth=0.75)
    
        ax0.set_title('Challenges of track {} by month'.format(df['title']))
        ax0.set_xlabel('Time (month)')
        ax0.set_ylabel('Number of challenge')
        ax1.set_ylabel('Prize per challenge')
        ax0.grid(True, axis='x')
        
        handle0, label0 = ax0.get_legend_handles_labels()
        handle1, label1 = ax1.get_legend_handles_labels()
        ax0.legend(handle0 + handle1, label0 + label1, loc=1, prop={'size': 8})
        
fig.tight_layout()